In [15]:
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from scipy.stats import pearsonr

In [2]:
embed = hub.load("universal-sentence-encoder_4")

2023-05-27 16:29:28.758986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
def on_bad_line(values):
    return values[:7]

columns_mapping = {
    0: 'genre',
    1: 'filename',
    2: 'year',
    3: 'index',
    4: 'score',
    5: 'sentence1',
    6: 'sentence2'
}

# quotings 3 = csv.QUOTE_NONE
train_df = pd.read_csv('Dataset/sts-train.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Train Dataframe {train_df.shape}')

test_df = pd.read_csv('Dataset/sts-test.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Test Dataframe {train_df.shape}')

shape of the Train Dataframe (5749, 7)
shape of the Test Dataframe (5749, 7)


In [17]:
y_pred = []
for s1, s2 in zip(test_df['sentence1'], test_df['sentence2']):
    embeddings = embed([s1, s2])
    simil = cosine_similarity(embeddings, embeddings)[0][1]
    y_pred.append(simil)

print(f'Pearson corr: {pearsonr(y_pred, test_df["score"])[0]}')

Pearson corr: 0.7872796725912385
